<div class="alert alert-info">

# Pymaceuticals Inc.


### Analysis
The drug company Pymaceuticals is testing drugs for treating squamous cell carcinoma (SCC), a commonly occurring form of skin cancer. We are looking at the data of 249 mice and they were administered 9 drug treatments and a placebo. Almost equal amount of male and female mice were tested.
After analysing the data we can conclude:-
* If we look at the summary_stats_tumor table, we can see that the drug Capomulin is more effective as compared to other drug regimens. The only other drug that is better than it is Ramicane.This is also evident by the box plots.
* The box plots also show that there are not many outliers in all the four drug regimens. 
* If we look at the first line plot of tumor volume vs. time point for a single mouse treated with Capomulin, we can see that first the tumor volume increases and then it starts decreasing.
* Finally if we look at the last two scatter plots, we can conclude that there is a positive correlation between average weight of mouse and average tumor volume(heavier the mouse bigger the tumor). The coefficient of correlation is 0.84 and it is quite high.</div>



In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single DataFrame
mouse_merged = pd.merge(mouse_metadata, study_results, how = 'right', on =['Mouse ID','Mouse ID'])
# Display the data table for preview
mouse_merged = mouse_merged[['Mouse ID', 'Timepoint', 'Tumor Volume (mm3)', 'Metastatic Sites', 
                               'Drug Regimen', 'Sex', 'Age_months', 'Weight (g)']]

mouse_merged.head()



In [ ]:
# Checking the number of mice.
mouse_merged['Mouse ID'].nunique()

In [ ]:
# Our data should be uniquely identified by Mouse ID and Timepoint
# Get the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
duplicate_mouse = mouse_merged.loc[mouse_merged.duplicated(subset = ['Mouse ID','Timepoint']),'Mouse ID'].unique()
duplicate_mouse

In [ ]:
# Optional: Get all the data for the duplicate mouse ID. 
duplicate_mouse_df = mouse_merged.loc[mouse_merged['Mouse ID'] == 'g989', :]
duplicate_mouse_df

In [ ]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
clean_df = mouse_merged.drop([107,137,329,360,620,681,815,869,950,1111,1195,1380,1592])
clean_df.head()


In [ ]:
# Checking the number of mice in the clean DataFrame.
clean_df['Mouse ID'].nunique()

# Summary Statistics

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 

clean_df_grouped = clean_df.groupby(['Drug Regimen'])

mean_drug_regimen = clean_df_grouped['Tumor Volume (mm3)'].mean()
median_drug_regimen = clean_df_grouped['Tumor Volume (mm3)'].median()
variance_drug_regimen = clean_df_grouped['Tumor Volume (mm3)'].var()
sd_drug_regimen = clean_df_grouped['Tumor Volume (mm3)'].std()
sem_drug_regimen = clean_df_grouped['Tumor Volume (mm3)'].sem()

# Assemble the resulting series into a single summary DataFrame.

summary_stats_tumor = pd.DataFrame({'Mean Tumor Volume':mean_drug_regimen, 'Median Tumor Volume':median_drug_regimen,
                                   'Tumor Volume Variance':variance_drug_regimen, 'Tumor Volume Std. Dev.':sd_drug_regimen,
                                   'Tumor Volume Std. Err.':sem_drug_regimen})
summary_stats_tumor

In [ ]:
# A more advanced method to generate a summary statistics table of mean, median, variance, standard deviation,
# and SEM of the tumor volume for each regimen (only one method is required in the solution)

# Using the aggregation method, produce the same summary statistics in a single line
summary_stats_tumor1 = clean_df.groupby(['Drug Regimen'])['Tumor Volume (mm3)'].aggregate(['mean','median','var','std','sem'])
summary_stats_tumor1

# Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using Pandas.
mouse_id = clean_df['Drug Regimen'].value_counts()
mouse_id
pandas_plot = mouse_id.plot( kind = 'bar',color = 'b')
plt.xlabel('Drug Regimen')
plt.ylabel('# of Observed Mouse Timepoints')




In [ ]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using pyplot.
fig, ax = plt.subplots()
plot = ax.bar(mouse_id.index, mouse_id.values, color ='b')
ax.set_xlabel('Drug Regimen')
ax.set_ylabel ('# of Observed Mouse Timepoints')
plt.xticks(rotation='vertical')
plt.show()


In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using Pandas
sex_count = clean_df['Sex'].value_counts()
sex_count
pie_pandas = sex_count.plot( kind = 'pie',autopct= "%1.1f%%")
plt.show()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot
labels = ['Male','Female']
fig,ax = plt.subplots()
ax.pie(sex_count, labels = labels, autopct = "%1.1f%%"  )
#plt.ylabel('Sex')
plt.axis('equal')
ax.set_ylabel('Sex')

plt.show()


# Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse
last_timepoint = clean_df.groupby(['Mouse ID'])['Timepoint'].max()
last_timepoint_df = pd.DataFrame(last_timepoint)
last_timepoint_df.head()





In [ ]:
# Merge this group df with the original DataFrame to get the tumor volume at the last timepoint

merged_timepoint = pd.merge(last_timepoint_df, clean_df, on = ('Mouse ID','Timepoint'))
merged_timepoint

In [ ]:
# Put treatments into a list for for loop (and later for plot labels)
treatments = ['Capomulin', 'Ramicane', 'Infubinol', 'Ceftamin']

# Create empty list to fill with tumor vol data (for plotting)
tumor_vol_data =[]

# Calculate the IQR and quantitatively determine if there are any potential outliers. 
for treatment in treatments:
    treatment_df = merged_timepoint.loc[merged_timepoint['Drug Regimen'] == treatment]
    tumor_vol = treatment_df['Tumor Volume (mm3)']
    tumor_vol_data.append(tumor_vol)
    quartiles = tumor_vol.quantile([.25,.5,.75])
    Quartile1 = quartiles[0.25]
    Quartile3 = quartiles[0.75]
    iqr = Quartile3 - Quartile1
    print(f'IQR for {treatment}: {iqr}')
    
    lower_bound = Quartile1 - (1.5*iqr)
    upper_bound = Quartile3 + (1.5*iqr)
    outliers = tumor_vol.loc[(treatment_df['Tumor Volume (mm3)'] > upper_bound) | 
                                        (treatment_df['Tumor Volume (mm3)'] < lower_bound)]
    print(f"{treatment}'s potential outlier {outliers}")
    



In [ ]:
# Generate a box plot that shows the distrubution of the tumor volume for each treatment group.

plt.boxplot(tumor_vol_data, flierprops = dict(markerfacecolor = 'r',marker ='o'))
plt.ylabel("Final Tumor Volume(mm3)")
plt.title('Final Tumor Volume by Drug')

plt.xticks([1,2,3,4],treatments)
plt.show()


# Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a single mouse treated with Capomulin

Capomulin_data = clean_df.loc[clean_df['Drug Regimen'] == 'Capomulin']
Capomulin_data = Capomulin_data.reset_index()
Capomulin_data
mouse_r944 = Capomulin_data.loc[Capomulin_data['Mouse ID'] == 'r944']
mouse_r944
# plot a line plot 
y = mouse_r944['Tumor Volume (mm3)']
x = mouse_r944['Timepoint']
plt.plot(x, y, marker = 'o')
plt.title('Capomulin Treatment of Mouse r944')
plt.xlabel('Timepoint(days)')
plt.ylabel('Tumor Volume (mm3)')
plt.show()

In [ ]:
# Generate a scatter plot of mouse weight vs. the average observed tumor volume for the entire Capomulin regimen
Capomulin_avg = Capomulin_data.groupby(['Mouse ID']).mean(numeric_only = True)
Capomulin_avg
plt.scatter(Capomulin_avg['Weight (g)'], Capomulin_avg['Tumor Volume (mm3)'])
plt.xlabel('Weight(g)')
plt.ylabel('Average Tumor Volume(mm3)')
plt.title('Average Tumor Volume by Weight')
plt.show()

# Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and a linear regression model 
# for mouse weight and average observed tumor volume for the entire Capomulin regimen

x = Capomulin_avg["Weight (g)"]
y = Capomulin_avg["Tumor Volume (mm3)"]
corr = st.pearsonr(x,y)
#print the correlation coefficient
print(f'The correlation between mouse weight and the average tumor volume is {round(corr[0],2)}.')

# Calculate linear regression model
(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x,y)
regress_values = slope * x + intercept

#plot scatter plot
plt.scatter(x,y)
plt.plot(x,regress_values ,color ='r')
plt.xlabel('Weight(g)')
plt.ylabel('Average Tumor Volume(mm3)')
plt.title('Average Tumor Volume by Weight')
plt.show()
